# Boolean functions 2024 | Perceptron-from-scratch task

A perceptron with a single output neuron can be used to determine whether an n-dimensional Boolean function is linearly separable.

A Boolean function is defined by a set of inputs. In this task, you implement a computer program that determines whether a Boolean function is linearly separable or not, using a perceptron with activation function $g(b)=sgn(b)$, where

$$b = \sum\limits_{j=1}^n w_j x_j - \theta$$

[See Eq. (5.9) in the course book for the case n = 2 (arXiv)](https://arxiv.org/abs/1901.05639).

Using this program, sample an n-dimensional Boolean function randomly and determine whether it is linearly separable.

Do this $10^4$ times for n = 2, 3, 4 and 5 dimensions, and save the number of linearly separable Boolean functions found.

The learning rules for the weights $wj$ and threshold $\theta$ are

$$\delta w_j^{(\mu)} = \eta(t^{(\mu)} - O^{(\mu)})x_j^{(\mu)},$$

and

$$\delta \theta^{(\mu)} = -\eta(t^{(\mu)} - O^{(\mu)})$$

See also Eq. (5.18)

For each boolean function, train the perceptron for 20 training epochs (update parameters once for every input-output pair) using a learning rate $\eta = 0.05$.

Initialize the weights from a normal distribution with mean 0 and varance 1/n, and initialize the threshold to 0.

Be sure to not count the same Boolean function twice. This can, for example, be done by adding every sampled Boolean function to a list and excluding the function if it comes up a second time.

---

#### Implementing a solution in Python

|    |    |
| -- | -- |
|$\delta w_j^{(\mu)}$ | change in the j:th weight for the μ:th training example |
|$\eta$ | learning rate |
|$t^{(\mu)}$ | target (correct) output for the μ:th training example |
|$O^{(\mu)}$ | the actual output produced by the perceptron for the μ:th training example |
|$x_j^{(\mu)}$ | j:th input feature for the μ:th training example |

In the Perceptron class, this is implemented as:
```
self.w += self.eta * (t - O) * x
```

Here, self.w is the weight vector, self.eta is η, t is the target output, O is the perceptron's output, and x is the input vector.

The threshold is very similarly updated:

```
self.theta -= self.learning_rate * (t - o)
```

#### 0. Generating random boolean functions

In [1]:
import numpy as np
from itertools import product

def random_bool(n):
    X = np.array(list(product([0, 1], repeat=n)))
    y = np.random.randint(0, 2, size=2**n)
    return X, y

The total number of possible functions grows exponentially ($2^{(2^n)}$)

- Takes an integer n as input, representing the number of input variables (or dimensions).
- Creates a numpy array X with all possible combinations of n binary values (0 or 1) using itertools.product() for finding the Cartesian Product
- Generates a random binary vector y of length $2^n$, representing the function's output for each input combination.

Example:

In [2]:
random_bool(2)

(array([[0, 0],
        [0, 1],
        [1, 0],
        [1, 1]]),
 array([0, 1, 1, 0]))

In the above example, X is our input data  with 4 samples and two features, while y is our binary label.

#### 1. Creating a perceptron

In [3]:
class Perceptron:
    def __init__(self, n_inputs, epochs=20):
        self.weights = np.random.normal(0, np.sqrt(1/n_inputs), n_inputs)
        self.theta = 0
        self.epochs = epochs
        self.learning_rate = 0.05

    def predict(self, X):
        return (np.dot(X, self.weights) - self.theta > 0).astype(int)

    def train(self, X, y):
        for _ in range(self.epochs):
            for x, t in zip(X, y):
                o = self.predict(x)
                self.weights += self.learning_rate * (t - o) * x
                self.theta -= self.learning_rate * (t - o)
        return np.all(self.predict(X) == y)

A Perceptron instance is initialized with random weights from a normal distribution, a threshold of 0, training epochs of 20, and a learning rate of 0.05.

The `predict` method is used during training to calculate the dot product of inputs and weights with subtracted threshold. It returns 1 if the result is positive.

The `train` method iterates over epochs, calls the prediction and adjusts weights and theta.

In [4]:
def is_linearly_separable(X, y, epochs=20):
    perceptron = Perceptron(X.shape[1], epochs=epochs)
    is_separable = perceptron.train(X, y)
    return is_separable

Example:

In [5]:
X, y = random_bool(2)

print(X, y)
print(f"\n{is_linearly_separable(X, y)=}")

[[0 0]
 [0 1]
 [1 0]
 [1 1]] [0 1 0 0]

is_linearly_separable(X, y)=True


Now we need to make sure we set our sample rate ($10^4$) and avoid duplicate function checks which is likely to be happening due to the *stochastic* `random_bool()`

#### 2. Iterative training over n_dimensions 

We do this by nesting loops, yeah.

In [6]:
def main(n_dimensions, n_samples = 10**4, epochs=20):
    results = {}
    for n in n_dimensions:
        separable_count, seen_functions = 0, set()
        for i in range(n_samples):
            X, y = random_bool(n)
            function_hash = hash(tuple(y))
            if function_hash not in seen_functions:
                seen_functions.add(function_hash)
                if is_linearly_separable(X, y, epochs):
                    separable_count += 1
        results[n] = separable_count
    return results

In [7]:
dimensions = [2,3,4,5]
results = main(dimensions)

for n, count in results.items():
    print(f"Linearly separable in Dimension {n}: {count}/10000")

Linearly separable in Dimension 2: 12/10000
Linearly separable in Dimension 3: 95/10000
Linearly separable in Dimension 4: 243/10000
Linearly separable in Dimension 5: 1/10000


#### Results

The perceptron drastically drops in performance past the 4th dimension. I ran the code a couple of times and often I found 0 possible linear separations. Only once did I find a single positive. But it's to no surprise given we only sample 10,000 (without replacement).

There's no guarantee to find *all* linearly separable functions unless we completely search the entire space.
If we really wanted to find all, we should consider removing sample randomization and explore a more systematic subset of the function space.

For R <= 4 the Perceptron class does a fairly good job.

What if we increase our sample rate and our training epochs?

In [8]:
results = main([5], 40_000, 80)
for n, count in results.items():
    print(f"Linearly separable in Dimension {n}: {count}/40,000")

Linearly separable in Dimension 5: 0/40,000


Never lucky. What about the lower dimensions?

In [11]:
results = main([2,3,4], 40_000, 80)
for n, count in results.items():
    print(f"Linearly separable in Dimension {n}: {count}/40,000")

Linearly separable in Dimension 2: 14/40,000
Linearly separable in Dimension 3: 104/40,000
Linearly separable in Dimension 4: 879/40,000


Nice.

#### Conclusion

With the initial hyperparameters tuned to 20 epochs and 1,000 input samples, this perceptron successfully found some, but not all, linear separations of boolean functions in lower dimensions. However, when trained on 40,000 samples and 80 epochs, we succesfully identified all valid 2- and 3-dimensional separations and half of the 4-dimensional ones. As there are $2(2^4)$ (65,536) 4-dimensional boolean functions and $2(2^5)$ (4,294,967,296) 5-dimensional ones, we shouldn't expect any better results without refactoring the `random_bool`, or brute forcing over $2(2^5)$ samples, hoping to get lucky.